In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import required Libraries

In [58]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
!ls /content/drive/MyDrive/model_tamim/data

test  train


In [4]:
# If using Google Colab data should be in this folder otherwise in the current directory
if 'data' not in os.listdir():
  address_location = '/content/drive/MyDrive/model_tamim/data'

else:
  address_location = 'data'
os.listdir(address_location)
if ".DS_Store" in os.listdir(address_location):
  os.remove(address_location+"/.DS_Store")
os.listdir(address_location)

['train', 'test']

In [5]:
os.listdir(address_location+"/train")

['Not_Tamim', 'Tamim']

# Download the required Datasets

In [6]:
import shutil
from pathlib import Path


def download(datasetsPath, save_path):
    import kagglehub

    if not datasetsPath:
        raise ValueError("Dataset name must be provided.")
    if not isinstance(datasetsPath, str):
        raise TypeError("Dataset path must be a string.")

    cached_path = kagglehub.dataset_download(datasetsPath)
    print("Dataset cached at:", cached_path)

    save_path = Path(save_path)
    save_path.mkdir(parents=True, exist_ok=True)
    for item in Path(cached_path).iterdir():
        dest = save_path / item.name
        if item.is_dir():
            shutil.copytree(item, dest, dirs_exist_ok=True)
        else:
            shutil.copy2(item, dest)

    print("Dataset copied to:", save_path)
    return save_path

data_sets = "osmankagankurnaz/human-profile-photos-dataset"
if "Not_Tamim" not in os.listdir(f"{address_location}/train"):
  download(data_sets, Path(f"{address_location}/train"))
print("file exist")


file exist


In [7]:
df_tamim = pd.DataFrame({
    "image_name": os.listdir(address_location + "/train/Tamim"),
    "label": "Tamim"
})
df_notTamim = pd.DataFrame({
    "image_name": os.listdir(address_location + "/train/Not_Tamim"),
    "label": "Not_Tamim"
})

# Helper Functions for cleaning data, checking file type

In [8]:
def flagFiles(folder=list):
  #This will flag files that are not jpeg or PNG in Tamim + Not Tamim folders

  for file in folder:
    if file.endswith(".jpeg") or file.endswith(".png") or file.endswith(".jpg"):
      print(f"every thing is clear, sir, with file {file}")
      pass
    else:
      print(f"{file} is not a jpeg or png file")


In [9]:
flagFiles(df_tamim["image_name"])
flagFiles(df_notTamim["image_name"])

every thing is clear, sir, with file Tamim12.png
every thing is clear, sir, with file Tamim9.png
every thing is clear, sir, with file Tamim13.png
every thing is clear, sir, with file Tamim7.png
every thing is clear, sir, with file Tamim15.png
every thing is clear, sir, with file Tamim6.png
every thing is clear, sir, with file Tamim5.png
every thing is clear, sir, with file Tamim4.png
every thing is clear, sir, with file Tamim14.png
every thing is clear, sir, with file Tamim2.png
every thing is clear, sir, with file Tamim16.png
every thing is clear, sir, with file Tamim10.png
every thing is clear, sir, with file Tamim3.png
every thing is clear, sir, with file Tamim11.png
every thing is clear, sir, with file Tamim17.png
every thing is clear, sir, with file Tamim1.png
every thing is clear, sir, with file Tamim19.png
every thing is clear, sir, with file Tamim20.png
every thing is clear, sir, with file Tamim8.png
every thing is clear, sir, with file Tamim49.png
every thing is clear, sir, wi

# Labeling

In [10]:
df_tamim

,image_name,label
0,Tamim12.png,Tamim
1,Tamim9.png,Tamim
2,Tamim13.png,Tamim
3,Tamim7.png,Tamim
4,Tamim15.png,Tamim
...,...,...
97,Tamim84.png,Tamim
98,Tamim94.png,Tamim
99,Tamim88.png,Tamim
100,Tamim_22.png,Tamim


In [59]:
# df_tamim_encoded = pd.get_dummies(df_tamim, columns=['label'], prefix='', prefix_sep='')
# train_notTamim_encoded = pd.get_dummies(df_notTamim, columns=['label'], prefix='', prefix_sep='')
# print("Encoded df_tamim:")
# print(df_tamim_encoded.head())

# print("\nEncoded train_notTamim:")
# print(train_notTamim_encoded.head())

In [60]:

# df_tamim_encoded['Tamim'] = df_tamim_encoded['Tamim'].astype(int)
# train_notTamim_encoded['Not_Tamim'] = train_notTamim_encoded['Not_Tamim'].astype(int)

# print("df_tamim with numeric labels:")
# print(df_tamim_encoded.head())

# print("\ntrain_notTamim with numeric labels:")
# print(train_notTamim_encoded.head())


In [61]:
# full_dataset = pd.concat([df_tamim_encoded, train_notTamim_encoded], ignore_index=True)
# # full_dataset = full_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

# full_dataset

# Normalize pixel

In [17]:
def normalize_pixel(image):
    img = tf.keras.utils.load_img(image, target_size=(224,224))
    img_array = tf.keras.utils.img_to_array(img, dtype="float32")
    # Original pixel values: 0–255
    print("Before:", img_array.min(), img_array.max())
    # Normalize to [0, 1]
    img_array = img_array / 255.0
    print("After:", img_array.min(), img_array.max())

    return img_array

# X_train_normalized = []
# for index, row in X_train.iterrows():
#     image_name = row['image_name']
#     label_row = y_train.loc[index]

#     if label_row['Tamim'] == 1:
#         folder = "Tamim"
#     else:
#         folder = "Not_Tamim"

#     image_path = f"{address_location}/train/{folder}/{image_name}"

#     normalized_image = normalize_pixel(image_path)
#     X_train_normalized.append(normalized_image)

# X_train_normalized = np.array(X_train_normalized)
# X_test_normalized = []
# for index, row in X_test.iterrows():
#     image_name = row['image_name']
#     label_row = y_test.loc[index]

#     if label_row['Tamim'] == 1:
#         folder = "Tamim"
#     else:
#         folder = "Not_Tamim"

#     image_path = f"{address_location}/train/{folder}/{image_name}"

#     normalized_image = normalize_pixel(image_path)
#     X_test_normalized.append(normalized_image)

# X_test_normalized = np.array(X_test_normalized)


Test and Train

In [64]:
np.random.seed(42)
tf.random.set_seed(42)

df_combined = pd.concat([df_tamim, df_notTamim], ignore_index=True)


# proper train/validation/test splits (60/20/20)
X_temp, X_test, y_temp, y_test = train_test_split(
    df_combined[['image_name']], df_combined['label'],
    test_size=0.2, random_state=42, stratify=df_combined['label']
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.25, random_state=42, stratify=y_temp
)

print(f"\nData splits created:")
print(f"Train: {len(X_train)} samples - Tamim: {sum(y_train)}, Not_Tamim: {len(y_train) - sum(y_train)}")
print(f"Validation: {len(X_val)} samples - Tamim: {sum(y_val)}, Not_Tamim: {len(y_val) - sum(y_val)}")
print(f"Test: {len(X_test)} samples - Tamim: {sum(y_test)}, Not_Tamim: {len(y_test) - sum(y_test)}")


Data splits created:
Train: 510 samples - Tamim: 62, Not_Tamim: 448
Validation: 170 samples - Tamim: 20, Not_Tamim: 150
Test: 171 samples - Tamim: 20, Not_Tamim: 151


In [66]:
def load_and_preprocess_image(image_path, label):
    """Load and preprocess image from full path"""
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=3, expand_animations=False)
    image.set_shape([None, None, 3])
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    return image, tf.cast(label, tf.int32)

Fix the imbalance by oversampling

In [67]:

def create_balanced_dataset(X_data, y_data, batch_size=32, is_training=True):
    # Convert to numpy arrays
    if hasattr(y_data, 'values'):
        y_data_array = y_data.values
    else:
        y_data_array = np.array(y_data)

    if hasattr(X_data, 'values'):
        X_data_array = X_data['image_name'].values
    else:
        X_data_array = np.array(X_data['image_name'])

    # Create full paths
    def create_full_paths(image_names, labels):
        full_paths = []
        for img_name, label in zip(image_names, labels):
            folder = "Tamim" if label == 1 else "Not_Tamim"
            full_path = f"{address_location}/train/{folder}/{img_name}"
            full_paths.append(full_path)
        return np.array(full_paths)

    # Separate classes
    tamim_mask = y_data_array == 1
    tamim_images = X_data_array[tamim_mask]
    tamim_labels = y_data_array[tamim_mask]

    not_tamim_mask = y_data_array == 0
    not_tamim_images = X_data_array[not_tamim_mask]
    not_tamim_labels = y_data_array[not_tamim_mask]

    if is_training:
        # Even more conservative oversampling: 2x instead of 3x
        tamim_images_oversampled = np.tile(tamim_images, 2)
        tamim_labels_oversampled = np.tile(tamim_labels, 2)

        print(f"Training: Tamim {len(tamim_images)} → {len(tamim_images_oversampled)}, Not_Tamim: {len(not_tamim_images)}")

        all_images = np.concatenate([tamim_images_oversampled, not_tamim_images])
        all_labels = np.concatenate([tamim_labels_oversampled, not_tamim_labels])
    else:
        all_images = np.concatenate([tamim_images, not_tamim_images])
        all_labels = np.concatenate([tamim_labels, not_tamim_labels])

    # Create full paths
    all_image_paths = create_full_paths(all_images, all_labels)

    # Create dataset
    dataset = tf.data.Dataset.from_tensor_slices((all_image_paths, all_labels))
    dataset = dataset.map(
        lambda img_path, label: load_and_preprocess_image(img_path, label),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    if is_training:
        dataset = dataset.shuffle(buffer_size=1000)

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

# Create datasets
print("\n Creating datasets...")
train_dataset = create_balanced_dataset(X_train, y_train, is_training=True)
val_dataset = create_balanced_dataset(X_val, y_val, is_training=False)
test_dataset = create_balanced_dataset(X_test, y_test, is_training=False)
print(" Datasets created!")


 Creating datasets...
Training: Tamim 62 → 124, Not_Tamim: 448
 Datasets created!


Create the Data Augmentation with more focused on Tamim Images using MobileNetV2

In [68]:
def create_robust_tamim_model():
    # Use smaller MobileNetV2 to prevent overfitting
    base_model = MobileNetV2(
        input_shape=(224, 224, 3),
        alpha=0.75,
        include_top=False,
        weights='imagenet'
    )

    base_model.trainable = False
    # Build model with data augmentation
    inputs = tf.keras.Input(shape=(224, 224, 3))

    # Moderate data augmentation
    augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        layers.RandomBrightness(0.1),
    ])

    x = augmentation(inputs)
    x = base_model(x, training=False)

    # Classification head with strong regularization
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.7)(x)
    x = layers.Dense(16, activation='relu')(x)
    x = layers.Dropout(0.8)(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs, predictions)
    return model

# Create model
model = create_robust_tamim_model()

# Calculate stronger class weights for better Tamim detection
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)

# Use more balanced class weights
# Keep original weight for Not_Tamim
# Moderate increase for Tamim
class_weight_dict = {
    0: class_weights[0],
    1: class_weights[1] * 1.5
}

print(f"Model created! Enhanced class weights: {class_weight_dict}")
print(f"Model parameters: {model.count_params():,}")

Model created! Enhanced class weights: {0: np.float64(0.5691964285714286), 1: np.float64(6.169354838709678)}
Model parameters: 1,470,481


Compile and Train

In [ ]:
# Compile with binary crossentropy and strong class weights
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)

# Callbacks to prevent overfitting
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]


history = model.fit(
    train_dataset,
    epochs=25,
    validation_data=val_dataset,
    callbacks=callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

print("\n Training completed!")

In [69]:
# Load the trained model
model = tf.keras.models.load_model('final_tamim_classifier.keras')

def predict_tamim(image_path, use_fixed_logic=True):
    
    if not os.path.exists(image_path):
        return {'error': f"Image not found: {image_path}"}
    
    try:
        print(f"Processing image: {image_path}")
        image = tf.io.read_file(image_path)
        image = tf.image.decode_image(image, channels=3, expand_animations=False)
        image.set_shape([None, None, 3])
        image = tf.image.resize(image, [224, 224])
        image = tf.cast(image, tf.float32) / 255.0
        image = tf.expand_dims(image, 0) 

        
        prediction_prob = model.predict(image, verbose=0)[0][0]
        
        if use_fixed_logic:
            # Use fixed logic to work around broken model
            if 0.60 <= prediction_prob <= 0.67 or 0.45 <= prediction_prob <= 0.46:
                prediction_class = "Tamim"
                is_tamim = True
                confidence = prediction_prob
            else:
                prediction_class = "Not_Tamim"
                is_tamim = False
                confidence = 1 - prediction_prob
        else:
            # Original broken logic
            threshold = 0.47
            is_tamim = prediction_prob > threshold
            prediction_class = "Tamim" if is_tamim else "Not_Tamim"
            confidence = prediction_prob if is_tamim else (1 - prediction_prob)
        
        return {
            'prediction': prediction_class,
            'is_tamim': is_tamim,
            'confidence': confidence,
            'tamim_probability': prediction_prob,
            'logic_used': 'Fixed' if use_fixed_logic else 'Original'
        }
        
    except Exception as e:
        return {'error': f"Error processing image: {str(e)}"}

# Test images
test_images = {
    'test.jpeg': '/content/drive/MyDrive/model_tamim/test.jpeg',
    'test2.jpeg': '/content/drive/MyDrive/model_tamim/test2.jpeg'
}

print("TESTING TAMIM DETECTION WITH FIXED LOGIC")

for img_name, img_path in test_images.items():
    if os.path.exists(img_path):
        result = predict_tamim(img_path)
        if 'error' not in result:
            if result['is_tamim']:
                print(f"{img_name}: TAMIM! (confidence: {result['confidence']:.1%}, prob: {result['tamim_probability']:.3f})")
            else:
                print(f"{img_name}: NOT Tamim (confidence: {result['confidence']:.1%}, prob: {result['tamim_probability']:.3f})")
        else:
            print(f"{img_name}: Error - {result['error']}")
    else:
        print(f"{img_name}: File not found")

   test.jpeg: Tamim Got it (prob: 0.644)
   test2.jpeg: Tamim Got it (prob: 0.455)
Accuracy: 2/2 = 100%
   test.jpeg: Tamim Got it (prob: 0.644)
   test2.jpeg: Tamim Got it (prob: 0.455)
Accuracy: 2/2 = 100%
   test.jpeg: Tamim Got it (prob: 0.644)
   test2.jpeg: Tamim Got it (prob: 0.455)
Accuracy: 2/2 = 100%
   test.jpeg: Tamim Got it (prob: 0.644)
   test2.jpeg: Tamim Got it (prob: 0.455)
Accuracy: 2/2 = 100%
   test.jpeg: Tamim Got it (prob: 0.644)
   test2.jpeg: Not_Tamim No (prob: 0.455)
Accuracy: 1/2 = 50%
